In [ ]:
from pyspark import SparkContext, StorageLevel
from pyspark.sql import SQLContext, functions
import sys
import json
import csv
import itertools
from time import time
import math
import pandas as pd

In [ ]:
def process(entry):
    revisedEntries= entry[0].replace('\'', '').split(',')
    return (revisedEntries[1], str(revisedEntries[0]), str(revisedEntries[2]),str(revisedEntries[3]), str(revisedEntries[5]))

In [ ]:
SparkContext.setSystemProperty('spark.executor.memory', '8g')
SparkContext.setSystemProperty('spark.driver.memory', '8g')
SparkContext.setSystemProperty('spark.driver.maxResultSize','1g')
sc = SparkContext('local[*]', 'task1')
sqlContext = SQLContext(sc)

In [ ]:
# The following pararmeters are configurable and can be changed as per the business requirements
input_file_path = ".\preometheus-sample.csv"
son_results_path = ".\kiana-match-results.txt"
output_file_path = ".\kiana-matched-records.csv"
mapid_map_path = ".\macid-map-john.txt"

In [ ]:
global_macid_dict = {}
global_macid_revers_dict = {}

with open(mapid_map_path, "r") as fp:
    global_macid_dict = json.load(fp)

for key, val in global_macid_dict.items():
    global_macid_revers_dict[val] = key

In [ ]:
df = sqlContext.read.csv(input_file_path, header=True)

matched_id_list =[]
with open(son_results_path, 'r') as f:
    reader = csv.reader(f)
    matched_id_list = list(reader)

matched_id_set = set([item for sublist in matched_id_list for item in sublist])

In [ ]:
finalRdd = df.rdd
finalRdd = finalRdd\
    .filter(lambda entry: global_macid_revers_dict[int(entry["MacId"])] in matched_id_set)

results = finalRdd.toDF().toPandas()
results['MacId'] = pd.to_numeric(results.MacId, errors='coerce')
results = results.replace({'MacId': global_macid_revers_dict})

results.to_csv(output_file_path, index=False)